# Введение в MapReduce модель на Python


In [4]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [5]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [6]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [7]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [8]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [9]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [11]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [12]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [13]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [14]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [15]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [16]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [17]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [18]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.6886672518073076),
 (1, 2.6886672518073076),
 (2, 2.6886672518073076),
 (3, 2.6886672518073076),
 (4, 2.6886672518073076)]

## Inverted index

In [19]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [20]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [21]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [22]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('what', 10)])]

## TeraSort

In [23]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.05574531925889403),
   (None, 0.0709116627512777),
   (None, 0.1330762135983259),
   (None, 0.13735324951888994),
   (None, 0.20347719807137687),
   (None, 0.22904632819603288),
   (None, 0.24925559876249204),
   (None, 0.29201850772054894),
   (None, 0.34573281919770127),
   (None, 0.38384926066232794),
   (None, 0.4115925011241267),
   (None, 0.414754267930428),
   (None, 0.4211751919075759),
   (None, 0.427054711202308),
   (None, 0.44045518944515827),
   (None, 0.47499337605057834)]),
 (1,
  [(None, 0.5437490360823528),
   (None, 0.5597266510677373),
   (None, 0.6174436510470236),
   (None, 0.6246386573388104),
   (None, 0.7705985166859597),
   (None, 0.7738626876902329),
   (None, 0.7763665009170431),
   (None, 0.7919254502865267),
   (None, 0.8315808266849622),
   (None, 0.8590228600009139),
   (None, 0.8700368125598068),
   (None, 0.9339975659302967),
   (None, 0.9919980538699222),
   (None, 0.9929117900106771)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [24]:
import random

# Функция MAP для нахождения максимального значения в списке чисел
def MAP(numList):
    return max(numList)

# Функция REDUCE для нахождения максимального значения среди всех списков чисел
def REDUCE(numList):
    return max(numList)

# Функция RECORDREADER для генерации случайных чисел в заданном диапазоне
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Генерация списка случайных чисел
record = RECORDREADER(15)
print(record)

partCount = 5
# Разделение списка на части по 5 элементов
final = [record[d:d + partCount] for d in range(0, len(record), partCount)]
print(final)

# Нахождение максимального значения среди всех частей списка
print(REDUCE(list(map(lambda x: MAP(x), final))))

[50, 59, 87, 34, 98, 22, 40, 30, 5, 36, 41, 38, 73, 80, 89]
[[50, 59, 87, 34, 98], [22, 40, 30, 5, 36], [41, 38, 73, 80, 89]]
98


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [25]:
from typing import Iterator, NamedTuple

# Функция MAP для создания пары (1, num)
def MAP(num):
    return (1, num)

# Функция REDUCE для вычисления среднего значения из итератора NamedTuple
def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

# Функция RECORDREADER для генерации списка случайных чисел
def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

# Группировка по ключу и преобразование в список
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
print(shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)


[(1, 41), (1, 58), (1, 55), (1, 74), (1, 66), (1, 74), (1, 40), (1, 64), (1, 38), (1, 66), (1, 82), (1, 69), (1, 36), (1, 73), (1, 64), (1, 99), (1, 9), (1, 77), (1, 90), (1, 25), (1, 91), (1, 50), (1, 78), (1, 58), (1, 28), (1, 52), (1, 34), (1, 92), (1, 2), (1, 59), (1, 27), (1, 25), (1, 39), (1, 54), (1, 93), (1, 66), (1, 40), (1, 93), (1, 73), (1, 86), (1, 54), (1, 45), (1, 51), (1, 100), (1, 23), (1, 60), (1, 88), (1, 67), (1, 88), (1, 95), (1, 82), (1, 85), (1, 79), (1, 29), (1, 51), (1, 45), (1, 7), (1, 33), (1, 10), (1, 83), (1, 40), (1, 87), (1, 60), (1, 21), (1, 23), (1, 69), (1, 12), (1, 96), (1, 80), (1, 27), (1, 51), (1, 67), (1, 43), (1, 10), (1, 38), (1, 14), (1, 30), (1, 40), (1, 25), (1, 4), (1, 88), (1, 3), (1, 98), (1, 52), (1, 26), (1, 87), (1, 61), (1, 28), (1, 38), (1, 85), (1, 53), (1, 33), (1, 36), (1, 80), (1, 23), (1, 59), (1, 52), (1, 72), (1, 0), (1, 55)]
[(1, [41, 58, 55, 74, 66, 74, 40, 64, 38, 66, 82, 69, 36, 73, 64, 99, 9, 77, 90, 25, 91, 50, 78, 58, 28,

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [26]:

# Функция MAP для создания пары (1, num)
def MAP(num):
    return (1, num)

# Функция REDUCE для вычисления среднего значения из итератора NamedTuple
def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

# Функция RECORDREADER для генерации списка случайных чисел
def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

# Функция для группировки значений по ключу
def group_by_key(iterable):
    grouped_data = {}
    for key, value in sorted(iterable, key=lambda x: x[0]):
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

# Группировка по ключу и преобразование в список
shuffle_output = group_by_key(map_output)
print(shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)


[(1, 33), (1, 24), (1, 51), (1, 42), (1, 0), (1, 78), (1, 49), (1, 49), (1, 32), (1, 86), (1, 38), (1, 22), (1, 60), (1, 73), (1, 36), (1, 2), (1, 6), (1, 36), (1, 61), (1, 24), (1, 58), (1, 49), (1, 37), (1, 83), (1, 17), (1, 63), (1, 43), (1, 85), (1, 45), (1, 99), (1, 14), (1, 23), (1, 23), (1, 78), (1, 9), (1, 48), (1, 12), (1, 11), (1, 48), (1, 89), (1, 70), (1, 79), (1, 78), (1, 76), (1, 11), (1, 10), (1, 8), (1, 99), (1, 74), (1, 80), (1, 16), (1, 17), (1, 85), (1, 34), (1, 74), (1, 95), (1, 43), (1, 85), (1, 59), (1, 89), (1, 63), (1, 74), (1, 69), (1, 31), (1, 57), (1, 48), (1, 87), (1, 96), (1, 14), (1, 82), (1, 26), (1, 74), (1, 23), (1, 90), (1, 68), (1, 58), (1, 29), (1, 23), (1, 76), (1, 46), (1, 33), (1, 97), (1, 29), (1, 76), (1, 79), (1, 96), (1, 33), (1, 73), (1, 87), (1, 50), (1, 13), (1, 27), (1, 10), (1, 72), (1, 46), (1, 92), (1, 91), (1, 70), (1, 79), (1, 44)]
[(1, [33, 24, 51, 42, 0, 78, 49, 49, 32, 86, 38, 22, 60, 73, 36, 2, 6, 36, 61, 24, 58, 49, 37, 83, 17, 6

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [27]:
def flatten(nested_iterable):
    # Функция для "выравнивания" вложенной структуры данных
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    # Группировка данных по ключу
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
    # Распределенная группировка данных по ключу
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    # Функция для определения разделителя (партиционера)
    global reducers
    return hash(obj) % reducers

# Функция для выполнения операций MapReduce
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):

    # Применяем функцию MAP к каждой записи во входных данных, затем "выравниваем" результаты
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    # Если задан комбайнер (COMBINER), применяем его к данным после группировки по ключу
    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    # Выполняем распределенную группировку данных по ключу с использованием заданного партиционера
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    # Применяем функцию REDUCE к каждой группе данных после группировки и "выравниваем" результаты
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                    reduce_partition[1]))), reduce_partitions)
       # Вычисляем общее количество отправленных по сети пар ключ-значение
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

    # Выводим информацию о количестве отправленных данных и возвращаем результат операций MapReduce
    print(f"{total_values_sent} key-value pairs were sent over a network.")
    return reduce_outputs



from typing import Iterator
import numpy as np

# Задание текстовых документов
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    # Форматирование входных данных для MapReduce
    global maps

    # Функция RECORDREADER принимает на вход раздел (split) данных и генерирует ключ-значение для каждой строки документа
    def RECORDREADER(split):
      # Перебираем каждый документ в разделе, присваивая ему уникальный идентификатор docid
        for (docid, document) in enumerate(split):
          # Для каждой строки в документе создаем уникальный идентификатор lineid и разбиваем строку по символу новой строки '\n'
            for (lineid, line) in enumerate(document.split('\n')):
               # Генерируем ключ в формате "docid:lineid" и значение - строку из документа
                yield ("{}:{}".format(docid, lineid), line)
    # Вычисляем размер раздела данных split_size как округленное вверх
    #значение от деления общего числа документов на количество задач MapReduce
    split_size = int(np.ceil(len(documents) / maps))
    # Цикл по документам с шагом split_size
    for i in range(0, len(documents), split_size):
      # Генерируем данные для каждого раздела данных путем вызова функции RECORDREADER с подмножеством документов
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
    # Функция Map для преобразования данных
    for word in line.split(" "):
        yield (word, word)

def REDUCE(key: str, value: Iterator[str]):
    # Функция Reduce для агрегации данных
    yield key

# Выполнение MapReduce операций с заданными функциями MAP и REDUCE
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output


56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'is', 'it']), (1, ['what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [28]:
from collections import defaultdict

# Функция для реализации MAP операции
def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C - выбираем только кортежи, удовлетворяющие условию
            mapped_result[t].append(t)
    return mapped_result.items()

# Функция для реализации REDUCE операции
def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)  # Выводим промежуточный результат
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])  # Добавляем первый элемент каждого кортежа в итоговый список
    return reduced_result

# Предикат для фильтрации кортежей с четным первым элементом
def C(t):
    return t[0] % 2 == 0

# Генерация случайных записей (кортежей формата (x, y, z))
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

# Создание набора записей
record = RECORDREADER(5)
print(record)

# Разбиение записей на равные части
part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

# Применение операции MAP и последующей REDUCE к разделенным записям
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))


[(49, 45, 23), (94, 74, 93), (21, 31, 29), (41, 12, 92), (6, 68, 10)]
[[(49, 45, 23), (94, 74, 93), (21, 31, 29), (41, 12, 92), (6, 68, 10)]]
[dict_items([((94, 74, 93), [(94, 74, 93)]), ((6, 68, 10), [(6, 68, 10)])])]
[(94, 74, 93), (6, 68, 10)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [29]:
S = set([25, 90, 69])  # Создаем множество S

def MAP(t):
    res_list = []
    for el in t:  # Перебираем элементы входного кортежа t
        if el in S:  # Проверяем, есть ли элемент el в множестве S
            res_list.append(el)  # Добавляем элемент el в результирующий список
    res = tuple(res_list)  # Преобразуем список в кортеж
    return (res, res)  # Возвращаем кортеж из результирующего списка дважды

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print(map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print(reduce_output)


[(79, 18, 83), (60, 49, 70), (85, 50, 80), (34, 97, 72), (75, 1, 24)]
[((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), (()

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [30]:
def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Функция REDUCE возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(5)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(100)
print(map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем функцию REDUCE к результату группировки
print(reduce_output)


[(3, 93, 79), (27, 35, 70), (67, 15, 62), (25, 26, 67), (2, 2, 91)]
[((5, 28, 9), (5, 28, 9)), ((55, 26, 13), (55, 26, 13)), ((44, 67, 87), (44, 67, 87)), ((70, 28, 32), (70, 28, 32)), ((12, 89, 99), (12, 89, 99)), ((85, 76, 95), (85, 76, 95)), ((48, 70, 60), (48, 70, 60)), ((35, 73, 64), (35, 73, 64)), ((96, 48, 99), (96, 48, 99)), ((75, 81, 28), (75, 81, 28)), ((64, 78, 97), (64, 78, 97)), ((27, 23, 50), (27, 23, 50)), ((47, 57, 92), (47, 57, 92)), ((81, 71, 99), (81, 71, 99)), ((35, 81, 90), (35, 81, 90)), ((67, 1, 22), (67, 1, 22)), ((83, 44, 42), (83, 44, 42)), ((85, 30, 17), (85, 30, 17)), ((49, 0, 1), (49, 0, 1)), ((50, 75, 62), (50, 75, 62)), ((45, 52, 50), (45, 52, 50)), ((78, 92, 36), (78, 92, 36)), ((83, 44, 35), (83, 44, 35)), ((47, 2, 35), (47, 2, 35)), ((67, 43, 94), (67, 43, 94)), ((64, 53, 0), (64, 53, 0)), ((11, 48, 82), (11, 48, 82)), ((15, 84, 73), (15, 84, 73)), ((7, 74, 66), (7, 74, 66)), ((39, 82, 34), (39, 82, 34)), ((95, 33, 100), (95, 33, 100)), ((90, 15, 99), 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [31]:
def MAP(t):
    return (t, t)  # Функция MAP возвращает кортеж из входного элемента и этого же элемента

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:  # Если количество значений равно 2
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]  # Генерирует список кортежей случайных чисел

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем элементы входного итерируемого объекта
        t[k2] = t.get(k2, []) + [v2]  # Группируем значения по ключу k2
    return t.items()  # Возвращаем список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерируем список кортежей случайных чисел
print(record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем функцию MAP ко всем элементам RECORDREADER(10)
print(map_output)

shuffle_output = group_by_key(map_output)  # Группируем значения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат в список
print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применяем функцию REDUCE к результату группировки
print(reduce_output)


[(2, 0), (0, 0), (3, 0), (2, 0), (3, 2), (2, 2), (0, 1), (1, 0), (0, 2), (1, 3), (1, 2), (0, 1), (2, 2), (1, 2), (2, 0), (3, 1), (2, 3), (1, 2), (1, 3), (3, 0), (1, 1), (3, 0), (0, 3), (1, 0), (2, 3), (3, 1), (1, 0), (0, 1), (3, 2), (3, 3), (3, 1), (0, 0), (0, 2), (2, 1), (2, 3), (3, 1), (1, 3), (3, 0), (0, 1), (1, 1), (2, 0), (3, 2), (0, 1), (1, 3), (2, 1), (1, 1), (2, 0), (2, 1), (2, 1), (2, 0), (3, 0), (1, 2), (1, 2), (0, 3), (1, 0), (2, 0), (3, 3), (0, 3), (0, 1), (1, 1), (3, 0), (2, 0), (2, 3), (1, 3), (1, 1), (1, 0), (3, 1), (2, 3), (0, 3), (0, 1), (1, 3), (1, 2), (1, 1), (0, 2), (3, 0), (3, 3), (2, 0), (2, 3), (2, 0), (0, 0), (0, 1), (3, 3), (1, 1), (1, 0), (1, 0), (0, 0), (1, 1), (1, 1), (1, 1), (1, 2), (3, 1), (1, 3), (0, 3), (2, 1), (1, 3), (1, 1), (1, 2), (1, 3), (2, 1), (1, 0)]
[((0, 0), (0, 0)), ((2, 0), (2, 0)), ((1, 3), (1, 3)), ((1, 1), (1, 1)), ((0, 3), (0, 3)), ((1, 2), (1, 2)), ((2, 0), (2, 0)), ((1, 0), (1, 0)), ((3, 1), (3, 1)), ((2, 3), (2, 3)), ((2, 0), (2, 0)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [32]:
rels = [1, 2]  # Инициализация массива rels с элементами 1 и 2

class Tuple:  # Определение класса Tuple для хранения данных и id представления
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])  # Генерация случайного кортежа данных
    rel_id = rels[random.randint(0, len(rels) - 1)]  # Выбор случайного id представления из массива rels
    return Tuple(data, rel_id)  # Возвращает экземпляр класса Tuple с данными и id представления

def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]  # Генерация списка случайных кортежей данных

def MAP(t: Tuple):
    return (t.data, t.rel_id)  # Функция MAP возвращает кортеж из данных и id представления

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:  # Если значение соответствует первому элементу в массиве rels
        return (key, key)  # Возвращает кортеж с ключом и значением равными ключу

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]  # Группировка значений по ключу k2
    return t.items()  # Возвращает список кортежей (ключ, список значений)

record = RECORDREADER(100)  # Генерация списка случайных кортежей данных

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу из RECORDREADER(100)
print(map_output)

shuffle_output = group_by_key(map_output)  # Группировка значений по ключу
shuffle_output = list(shuffle_output)  # Преобразование результата в список
print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]  # Применение функции REDUCE к результату группировки
print(reduce_output)


[(((0, 0), (2, 2), (0, 0)), 2), (((1, 3), (2, 1), (0, 2)), 2), (((2, 1), (2, 2), (3, 0)), 2), (((2, 1), (3, 1), (2, 1)), 1), (((0, 0), (2, 1), (0, 1)), 1), (((3, 1), (0, 0), (3, 2)), 1), (((1, 3), (2, 0), (2, 0)), 2), (((1, 0), (3, 3), (3, 3)), 1), (((0, 1), (2, 2), (1, 0)), 1), (((0, 1), (2, 0), (3, 3)), 1), (((0, 3), (3, 3), (0, 2)), 1), (((1, 2), (1, 3), (2, 1)), 2), (((3, 2), (2, 3), (3, 1)), 1), (((2, 0), (1, 1), (2, 1)), 1), (((1, 1), (3, 2), (0, 3)), 2), (((3, 0), (1, 2), (2, 1)), 1), (((2, 1), (3, 3), (1, 0)), 2), (((0, 2), (2, 0), (3, 2)), 1), (((1, 3), (2, 0), (1, 0)), 2), (((3, 0), (2, 1), (0, 0)), 2), (((1, 3), (1, 0), (3, 0)), 2), (((2, 2), (2, 2), (3, 3)), 2), (((1, 2), (3, 2), (2, 3)), 1), (((3, 3), (2, 0), (3, 2)), 1), (((1, 1), (2, 0), (1, 3)), 2), (((3, 1), (1, 3), (0, 2)), 1), (((2, 1), (1, 2), (2, 3)), 2), (((3, 1), (3, 1), (3, 2)), 1), (((2, 1), (0, 0), (3, 2)), 2), (((1, 3), (2, 3), (0, 3)), 2), (((1, 0), (0, 1), (2, 3)), 1), (((1, 0), (0, 2), (0, 3)), 1), (((0, 1

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [34]:
rels = [1, 2]  # Инициализация списка идентификаторов связей

class Tuple:  # Определение класса кортежа, хранящего данные и идентификатор связи из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():  # Функция для генерации случайного кортежа с данными и идентификатором связи
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования данных кортежа в соответствии с условием
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для объединения данных по ключу
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print(map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print(reduce_output)


[(2, (1, 1)), (2, (2, 3)), (2, (2, 2)), (1, (1, 1)), (2, (1, 0)), (1, (1, 0)), (1, (1, 2)), (3, (2, 3)), (1, (2, 2)), (3, (1, 2)), (3, (2, 0)), (3, (2, 3)), (0, (2, 1)), (1, (2, 3)), (0, (2, 2)), (2, (1, 2)), (1, (2, 1)), (2, (2, 1)), (0, (1, 3)), (3, (2, 2)), (2, (1, 2)), (2, (1, 0)), (3, (2, 0)), (0, (2, 1)), (3, (2, 3)), (2, (2, 2)), (0, (2, 1)), (2, (1, 0)), (0, (1, 2)), (3, (1, 0)), (0, (2, 3)), (0, (1, 2)), (3, (1, 2)), (2, (2, 1)), (0, (1, 3)), (3, (2, 1)), (1, (2, 3)), (2, (2, 0)), (0, (1, 1)), (0, (2, 3)), (0, (2, 2)), (2, (2, 1)), (0, (1, 1)), (2, (2, 1)), (3, (1, 1)), (1, (1, 3)), (1, (2, 0)), (0, (2, 3)), (0, (2, 2)), (2, (2, 0)), (3, (2, 2)), (1, (1, 1)), (0, (1, 1)), (0, (2, 3)), (2, (2, 0)), (3, (1, 1)), (2, (1, 0)), (1, (2, 0)), (2, (1, 1)), (3, (2, 2)), (2, (1, 0)), (1, (1, 2)), (1, (1, 0)), (0, (2, 1)), (0, (2, 3)), (2, (1, 0)), (3, (1, 2)), (1, (2, 2)), (1, (1, 0)), (2, (2, 3)), (2, (2, 1)), (0, (1, 0)), (2, (1, 0)), (1, (1, 0)), (0, (1, 3)), (1, (1, 1)), (1, (2, 3))

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [35]:
def get_random_tuple():  # Функция для генерации случайного кортежа с тремя случайными значениями от 0 до 3
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):  # Функция для преобразования входного кортежа, возвращающая первые два элемента кортежа
  return (t[0], t[1])

def tetta(values):  # Функция для вычисления суммы значений в переданном списке
  return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для вычисления суммы значений в переданных значениях и возврата пары (ключ, сумма)
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):  # Функция для группировки данных по ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)  # Создание списка случайных кортежей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применение функции MAP к каждому элементу
print(map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print(reduce_output)


[(1, 2), (1, 2), (1, 0), (3, 0), (0, 3), (0, 2), (0, 2), (2, 1), (0, 0), (3, 2), (2, 3), (1, 0), (2, 0), (2, 1), (2, 2), (2, 0), (3, 2), (3, 2), (2, 0), (0, 3), (1, 3), (2, 3), (3, 0), (2, 3), (3, 2), (1, 3), (1, 2), (0, 2), (1, 1), (0, 2), (3, 3), (3, 0), (0, 0), (1, 3), (1, 0), (3, 1), (1, 0), (0, 3), (1, 2), (0, 0), (1, 2), (2, 0), (2, 1), (0, 1), (3, 3), (0, 1), (3, 2), (3, 0), (0, 2), (0, 1), (2, 3), (3, 3), (1, 3), (2, 1), (2, 2), (3, 2), (3, 1), (1, 3), (0, 1), (1, 2), (2, 0), (1, 1), (3, 3), (2, 2), (3, 0), (3, 3), (1, 0), (3, 3), (1, 2), (0, 0), (3, 1), (0, 0), (3, 2), (1, 0), (2, 2), (2, 2), (1, 0), (3, 3), (0, 1), (2, 2), (3, 1), (2, 3), (1, 1), (0, 1), (0, 2), (2, 3), (3, 3), (3, 0), (2, 1), (0, 1), (3, 1), (2, 2), (1, 3), (3, 1), (3, 3), (1, 3), (0, 2), (2, 0), (3, 1), (0, 3)]
[(1, [2, 2, 0, 0, 3, 3, 2, 1, 3, 0, 0, 2, 2, 3, 3, 2, 1, 0, 2, 0, 0, 1, 3, 3]), (3, [0, 2, 2, 2, 0, 2, 3, 0, 1, 3, 2, 0, 3, 2, 1, 3, 0, 3, 3, 1, 2, 3, 1, 3, 0, 1, 1, 3, 1]), (0, [3, 2, 2, 0, 3, 2, 2,

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [39]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [40]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [41]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [42]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [45]:
import numpy as np  # Импорт библиотеки numpy под псевдонимом np
I = 2  # Задание значения переменной I равным 2
J = 3  # Задание значения переменной J равным 3
K = 4*10  # Задание значения переменной K равным 40
small_mat = np.random.rand(I,J)  # Создание случайной матрицы размером IxJ
big_mat = np.random.rand(J,K)  # Создание случайной матрицы размером JxK
reference_solution = np.matmul(small_mat, big_mat)  # Вычисление матричного произведения small_mat и big_mat

def RECORDREADER():  # Определение функции RECORDREADER без аргументов
  for i in range(small_mat.shape[0]):  # Цикл по строкам матрицы small_mat
    for j in range(small_mat.shape[1]):  # Цикл по столбцам матрицы small_mat
      yield ((0, i, j), small_mat[i,j])  # Генерация кортежа для первой матрицы

  for j in range(big_mat.shape[0]):  # Цикл по строкам матрицы big_mat
    for k in range(big_mat.shape[1]):  # Цикл по столбцам матрицы big_mat
      yield ((1, j, k), big_mat[j,k])  # Генерация кортежа для второй матрицы

def MAP_JOIN(k1, v1):  # Определение функции MAP_JOIN с ключем k1 и значением v1
  (mat_num, i, j) = k1  # Распаковка ключа
  w = v1  # Присвоение значения из значения
  if mat_num == 0:  # Если номер матрицы равен 0
    yield (j, (mat_num, i, w))  # Генерация значения для первой матрицы
  else:  # Иначе
    yield (i, (mat_num, j, w))  # Генерация значения для второй матрицы

def REDUCE_JOIN(key, values):  # Определение функции REDUCE_JOIN с ключем key и значениями values
  from_first_mat = [v for v in values if v[0] == 0]  # Фильтрация значений для первой матрицы
  from_second_mat = [v for v in values if v[0] == 1]  # Фильтрация значений для второй матрицы
  for f in from_first_mat:  # Цикл по элементам из первой матрицы
    for s in from_second_mat:  # Цикл по элементам из второй матрицы
      yield ((f[1], s[1]), f[2] * s[2])  # Генерация умноженных значений


def MAP_MUL(k1, v1):  # Определение функции MAP_MUL с ключем k1 и значением v1
  (i, k) = k1  # Распаковка ключа
  yield ((i, k), v1)  # Генерация кортежа с ключом (i, k) и значением v1

def REDUCE_MUL(key, values):  # Определение функции REDUCE_MUL с ключем key и значениями values
  res_el_value = 0  # Инициализация переменной для хранения результата
  for v in values:  # Цикл по значениям
    res_el_value += v  # Суммирование значений
  yield (key, res_el_value)  # Генерация кортежа с ключом key и суммой значений

def GET_JOINED():  # Определение функции GET_JOINED без аргументов
  for j in joined:  # Цикл по элементам из joined
    yield j  # Генерация элемента


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)  # Применение MapReduce к функциям RECORDREADER, MAP_JOIN и REDUCE_JOIN
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)  # Применение MapReduce к функциям GET_JOINED, MAP_MUL и REDUCE_MUL
np.allclose(reference_solution, asmatrix(solution))  # Проверка на равенство решения с эталонным значением и вывод результата (True/False)


True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [52]:
# Определение размеров матриц
I = 2
J = 3
K = 4*10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  # Разделение данных на части и применение MAP функции
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  # Применение COMBINER (если есть)
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  # Распределенная группировка данных по ключу (shuffle)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  # Применение REDUCE функции
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Преобразование результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

# Проверка результата на соответствие эталонному решению
np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.2537361864238587), ((0, 1), 0.22112861349117416), ((0, 2), 0.3558377106705493), ((0, 3), 0.17859496664487537), ((0, 4), 0.3485999444921442), ((0, 5), 0.20735754551562469), ((0, 6), 0.006672062688435702), ((0, 7), 0.21549615982385278), ((0, 8), 0.2981969500298829), ((0, 9), 0.046123607196346325), ((0, 10), 0.05169192255717443), ((0, 11), 0.1658368552316355), ((0, 12), 0.003934071715749017), ((0, 13), 0.10184886846039887), ((0, 14), 0.09270864527503789), ((0, 15), 0.03923937641781827), ((0, 16), 0.35808446043565156), ((0, 17), 0.19269698043918299), ((0, 18), 0.24915127982239696), ((0, 19), 0.06447635875373758), ((0, 20), 0.09979194932179429), ((0, 21), 0.15604023429641573), ((0, 22), 0.09524997045531196), ((0, 23), 0.007177136412836752), ((0, 24), 0.11384423002948471), ((0, 25), 0.13687301338112712), ((0, 26), 0.14008325781965403), ((0, 27), 0.3654564692339463), ((0, 28), 0.23597894611038717), ((0, 29), 0.2499986280560371), (

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [57]:
# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.03389989464459296), ((0, 1), 0.004199108980542996), ((0, 2), 0.06093059247082387), ((0, 3), 0.06638556467248702), ((0, 4), 0.10059834532561183), ((0, 5), 0.11183795780274888), ((0, 6), 0.06146085202440543), ((0, 7), 0.018193971786627624), ((0, 8), 0.08576939050226123), ((0, 9), 0.0816578390165226), ((0, 10), 0.0645913514082816), ((0, 11), 0.07164535998259489), ((0, 12), 0.05930517824400405), ((0, 13), 0.034867405715154925), ((0, 14), 0.04762068558743972), ((0, 15), 0.02682770798005675), ((0, 16), 0.0465977376086562), ((0, 17), 0.10204702214999932), ((0, 18), 0.09345157092942771), ((0, 19), 0.0414170704752795), ((0, 20), 0.01795162051019693), ((0, 21), 0.06928019449234497), ((0, 22), 0.07597215065519233), ((0, 23), 0.09190114819347231), ((0, 24), 0.045430086898166314), ((0, 25), 0.056063503943761264), ((0, 26), 0.10925158925896518), ((0, 27), 0.08164969917607867), ((0, 28), 0.07992519914177697), ((0, 29), 0.08243540186235923

True